# Imports

In [90]:
import pandas as pd
import numpy as np
import datetime as dt

# Load Data

In [31]:
period = pd.read_csv('/home/ka_wagon/code/karynagogunska/mood_prediction/mood_prediction/data/raw_data/My_Calendar-2021-10-06.txt',sep='\t',names=["date", "start_end"],index_col=False)
period

,date,start_end
0,"Feb 7, 2014",Weight:68.7kg
1,"Feb 9, 2014",Weight:68.9kg
2,"Feb 10, 2014",Weight:69.1kg
3,"Feb 13, 2014",Weight:68.1kg
4,"Feb 14, 2014",Weight:67.7kg
...,...,...
451,"Jul 27, 2021",Period Ends
452,"Aug 20, 2021",Period Starts
453,"Aug 24, 2021",Period Ends
454,"Sep 15, 2021",Period Starts


In [32]:
mood = pd.read_csv('/home/ka_wagon/code/karynagogunska/mood_prediction/mood_prediction/data/raw_data/daylio_export_2021_10_06.csv')
mood

,full_date,date,weekday,time,mood,activities,note_title,note
0,2021-10-06,6 octobre,mercredi,18:31,fine,sweets | pill | walking,NaN,NaN
1,2021-10-05,5 octobre,mardi,15:38,ok,coke | fast-foods | chocolate | pill,NaN,NaN
2,2021-10-04,4 octobre,lundi,15:26,ok,coke | sweets | pill,NaN,NaN
3,2021-10-03,3 octobre,dimanche,08:56,ok,coke | sweets | pill | walking,NaN,NaN
4,2021-10-02,2 octobre,samedi,20:37,ok,sweets | showering | pill | walking,NaN,NaN
...,...,...,...,...,...,...,...,...
832,2020-01-09,9 janvier,jeudi,11:15,so so,hard to wake up | sugar control | meditation,NaN,NaN
833,2020-01-08,8 janvier,mercredi,22:11,so so,alcohol | courses | family,NaN,NaN
834,2020-01-08,8 janvier,mercredi,19:29,so so,alcohol | movies | friends | doctor | cook...,NaN,NaN
835,2020-01-08,8 janvier,mercredi,11:24,bien,early wake up,NaN,NaN


# Period dataset 

## Explore

In [33]:
period.head()

,date,start_end
0,"Feb 7, 2014",Weight:68.7kg
1,"Feb 9, 2014",Weight:68.9kg
2,"Feb 10, 2014",Weight:69.1kg
3,"Feb 13, 2014",Weight:68.1kg
4,"Feb 14, 2014",Weight:67.7kg


In [34]:
period.columns

Index(['date', 'start_end'], dtype='object')

In [35]:
period.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       456 non-null    object
 1   start_end  456 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


In [36]:
period.describe()

,date,start_end
count,456,456
unique,445,55
top,"Nov 20, 2016","Medicine:Противозачаточные таблетки ludeal ge,"
freq,2,306


In [38]:
period['start_end'].nunique()

55

In [40]:
period['start_end'].unique()

array(['Weight:68.7kg', 'Weight:68.9kg', 'Weight:69.1kg', 'Weight:68.1kg',
       'Weight:67.7kg', 'Weight:69.3kg', 'Weight:68.8kg', 'Weight:68.5kg',
       'Weight:67.0kg', 'Weight:68.0kg', 'Weight:67.4kg', 'Weight:67.3kg',
       'Weight:67.2kg', 'Weight:66.7kg', 'Weight:65.3kg', 'Weight:65.8kg',
       'Weight:65.2kg', 'Weight:65.5kg', 'Weight:64.7kg', 'Weight:64.6kg',
       'Weight:66.5kg', 'Weight:66.2kg', 'Weight:66.4kg', 'Weight:66.0kg',
       'Weight:67.5kg', 'Medicine:Свеча,', 'Medicine:Гинофлекс,',
       'Temperature:37.2℃', 'Temperature:36.8℃', 'Temperature:36.5℃',
       'Temperature:36.2℃', 'Temperature:36.7℃', 'Temperature:36.9℃',
       'Temperature:36.6℃', 'Temperature:36.4℃', 'Temperature:36.0℃',
       'Weight:63.3kg', 'Weight:64.0kg', 'Weight:63.2kg', 'Weight:63.6kg',
       'Weight:63.4kg', 'Weight:63.5kg',
       'Moods:Unsafe,Tense,Impatient,Disappointed,Furious,Secretive',
       'Weight:63.0kg', 'Weight:62.8kg',
       'Medicine:Противозачаточные таблетки lud

## Clean before merge

In [45]:
#date column
period.date = pd.to_datetime(period.date)

In [57]:
period.date

0     2014-02-07
1     2014-02-09
2     2014-02-10
3     2014-02-13
4     2014-02-14
         ...    
451   2021-07-27
452   2021-08-20
453   2021-08-24
454   2021-09-15
455   2021-09-19
Name: date, Length: 456, dtype: datetime64[ns]

In [63]:
period.tail()

,date,start_end
451,2021-07-27,Period Ends
452,2021-08-20,Period Starts
453,2021-08-24,Period Ends
454,2021-09-15,Period Starts
455,2021-09-19,Period Ends


In [69]:
# first occurence of 'period starts'
temp_index = period.start_end[period.start_end == 'Period Starts'].first_valid_index()
temp_index

396

In [71]:
period_df = period.iloc[temp_index:]

In [81]:
period_df.reset_index(drop=True, inplace =True)

In [111]:
# fill in missing dates
dates = pd.date_range(start = period_df.date.min(), end = period_df.date.max())
period_df = period_df.set_index('date').reindex(dates).fillna(0.0).rename_axis('date').reset_index()

In [112]:
period_df

,date,start_end,period_day
0,2019-09-09,Period Starts,0.0
1,2019-09-10,0.0,0.0
2,2019-09-11,0.0,0.0
3,2019-09-12,0.0,0.0
4,2019-09-13,Period Ends,0.0
...,...,...,...
737,2021-09-15,Period Starts,0.0
738,2021-09-16,0.0,0.0
739,2021-09-17,0.0,0.0
740,2021-09-18,0.0,0.0


In [83]:
# add column of day of period (ordinal),e.g. 1st,2nd,3rd....26th..
period_df['period_day'] = 0

/tmp/ipykernel_18732/2485062785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_df['period_day'] = 0


Pseudo to add column of day of period (ordinal),e.g. 1st,2nd,3rd....26th..

 - check start_end value in each row
 - if value == period start -> start incrementing until value == period end
 - repeat when next value == period start is met

In [126]:
period_df.index[period_df.start_end == 'Period Starts']

Int64Index([  0,  26,  55,  81, 106, 134, 157, 182, 212, 237, 267, 292, 318,
            343, 368, 395, 419, 444, 471, 498, 522, 549, 575, 602, 628, 655,
            683, 711, 737],
           dtype='int64')

In [92]:
(period_df.date[1] - period_df.date[0]).days

4

In [128]:
from itertools import combinations

In [130]:
[i for i in combinations(period_df.index[period_df.start_end == 'Period Starts'].to_list(), 2)]

[(0, 26),
 (0, 55),
 (0, 81),
 (0, 106),
 (0, 134),
 (0, 157),
 (0, 182),
 (0, 212),
 (0, 237),
 (0, 267),
 (0, 292),
 (0, 318),
 (0, 343),
 (0, 368),
 (0, 395),
 (0, 419),
 (0, 444),
 (0, 471),
 (0, 498),
 (0, 522),
 (0, 549),
 (0, 575),
 (0, 602),
 (0, 628),
 (0, 655),
 (0, 683),
 (0, 711),
 (0, 737),
 (26, 55),
 (26, 81),
 (26, 106),
 (26, 134),
 (26, 157),
 (26, 182),
 (26, 212),
 (26, 237),
 (26, 267),
 (26, 292),
 (26, 318),
 (26, 343),
 (26, 368),
 (26, 395),
 (26, 419),
 (26, 444),
 (26, 471),
 (26, 498),
 (26, 522),
 (26, 549),
 (26, 575),
 (26, 602),
 (26, 628),
 (26, 655),
 (26, 683),
 (26, 711),
 (26, 737),
 (55, 81),
 (55, 106),
 (55, 134),
 (55, 157),
 (55, 182),
 (55, 212),
 (55, 237),
 (55, 267),
 (55, 292),
 (55, 318),
 (55, 343),
 (55, 368),
 (55, 395),
 (55, 419),
 (55, 444),
 (55, 471),
 (55, 498),
 (55, 522),
 (55, 549),
 (55, 575),
 (55, 602),
 (55, 628),
 (55, 655),
 (55, 683),
 (55, 711),
 (55, 737),
 (81, 106),
 (81, 134),
 (81, 157),
 (81, 182),
 (81, 212),
 (

# Mood dataset

## Explore

In [17]:
mood.head()

,full_date,date,weekday,time,mood,activities,note_title,note
0,2021-10-06,6 octobre,mercredi,18:31,fine,sweets | pill | walking,NaN,NaN
1,2021-10-05,5 octobre,mardi,15:38,ok,coke | fast-foods | chocolate | pill,NaN,NaN
2,2021-10-04,4 octobre,lundi,15:26,ok,coke | sweets | pill,NaN,NaN
3,2021-10-03,3 octobre,dimanche,08:56,ok,coke | sweets | pill | walking,NaN,NaN
4,2021-10-02,2 octobre,samedi,20:37,ok,sweets | showering | pill | walking,NaN,NaN


In [12]:
mood.columns

Index(['full_date', 'date', 'weekday', 'time', 'mood', 'activities',
       'note_title', 'note'],
      dtype='object')

In [19]:
mood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   full_date   837 non-null    object 
 1   date        837 non-null    object 
 2   weekday     837 non-null    object 
 3   time        837 non-null    object 
 4   mood        837 non-null    object 
 5   activities  689 non-null    object 
 6   note_title  0 non-null      float64
 7   note        0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 52.4+ KB


In [18]:
mood.describe()

,note_title,note
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [11]:
mood.mood.nunique()

9

In [22]:
mood.mood.unique()

array(['fine', 'ok', 'so so', 'mix', 'manie', 'bad', 'bien', 'awful ',
       'awesome '], dtype=object)

In [23]:
mood.weekday.nunique()

7

## Clean before merge

In [48]:
#date column
mood.full_date = pd.to_datetime(mood.full_date)

In [53]:
mood.drop(columns='date',inplace=True)

KeyError: "['date'] not found in axis"

In [54]:
mood.full_date

0     2021-10-06
1     2021-10-05
2     2021-10-04
3     2021-10-03
4     2021-10-02
         ...    
832   2020-01-09
833   2020-01-08
834   2020-01-08
835   2020-01-08
836   2020-01-07
Name: full_date, Length: 837, dtype: datetime64[ns]

In [60]:
mood.rename(columns={'full_date':'date'},inplace=True)

# Unite both in one

In [72]:
#pd.merge(mood, period_df, how='left',on="date")

,date,weekday,time,mood,activities,note_title,note,start_end
0,2021-10-06,mercredi,18:31,fine,sweets | pill | walking,NaN,NaN,NaN
1,2021-10-05,mardi,15:38,ok,coke | fast-foods | chocolate | pill,NaN,NaN,NaN
2,2021-10-04,lundi,15:26,ok,coke | sweets | pill,NaN,NaN,NaN
3,2021-10-03,dimanche,08:56,ok,coke | sweets | pill | walking,NaN,NaN,NaN
4,2021-10-02,samedi,20:37,ok,sweets | showering | pill | walking,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
832,2020-01-09,jeudi,11:15,so so,hard to wake up | sugar control | meditation,NaN,NaN,NaN
833,2020-01-08,mercredi,22:11,so so,alcohol | courses | family,NaN,NaN,NaN
834,2020-01-08,mercredi,19:29,so so,alcohol | movies | friends | doctor | cook...,NaN,NaN,NaN
835,2020-01-08,mercredi,11:24,bien,early wake up,NaN,NaN,NaN
